# QDET Analytics Module Tutorial

This notebook demonstrates all available tools in the QDET analytics module using the Iris dataset. The analytics module provides quantum and hybrid machine learning algorithms for classification, clustering, regression, anomaly detection, and more.

## 1. Import Required Libraries

We import pandas, numpy, matplotlib, seaborn, and all tools from the QDET analytics module.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import sys
import os
sys.path.append(os.path.abspath('..'))

# Import all analytics module tools
from qudet.analytics import (
    QuantumKernelAnomalyDetector,
    QuantumAutoencoder,
    QuantumSVC,
    QuantumKMeans,
    QuantumFeatureSelector,
    QuantumKernelRegressor,
    VariationalQuantumEigensolver,
    QuantumNeuralNetwork,
    QuantumEnsemble,
    QuantumTimeSeriesPredictor
)

# Set random seed for reproducibility
np.random.seed(42)

print("✓ All libraries and analytics tools imported successfully!")

## 2. Load and Explore the Iris Dataset

Load the iris.csv dataset and explore its structure, dimensions, and basic statistics.

In [ ]:
# Load the Iris dataset
iris_df = pd.read_csv('../qudet/datasets/iris.csv')

print("Dataset Shape:", iris_df.shape)
print("\nFirst 5 rows:")
print(iris_df.head())

print("\nData Types:")
print(iris_df.dtypes)

print("\nBasic Statistics:")
print(iris_df.describe())

print("\nMissing Values:")
print(iris_df.isnull().sum())

print("\nTarget Variable Distribution:")
print(iris_df['variety'].value_counts())

## 3. Quantum Kernel Anomaly Detector

**Description**: Detects anomalies by mapping data to a high-dimensional Quantum Hilbert Space using a quantum kernel. It feeds the precomputed quantum kernel into a One-Class SVM for anomaly detection.

**Use Case**: Identify unusual flower measurements that deviate from the normal pattern of their species.

In [ ]:
# Prepare data - use first 30 samples for demonstration
X_anomaly = iris_df.iloc[:30, :-1].values

# Initialize and fit the Quantum Kernel Anomaly Detector
qkad = QuantumKernelAnomalyDetector(n_qubits=4, nu=0.1)
qkad.fit(X_anomaly)

# Predict anomalies (-1 = anomaly, 1 = normal)
anomaly_predictions = qkad.predict(X_anomaly)

print("Anomaly Detection Results:")
print(f"Normal samples: {(anomaly_predictions == 1).sum()}")
print(f"Anomalies detected: {(anomaly_predictions == -1).sum()}")
print(f"\nFirst 10 predictions: {anomaly_predictions[:10]}")

## 4. Quantum Autoencoder

**Description**: A variational quantum autoencoder that compresses input data from N qubits into K latent qubits, discarding noise while preserving signal. Essential for reducing qubit requirements in downstream quantum tasks.

**Use Case**: Compress the 4-dimensional iris features into a lower-dimensional representation.

In [ ]:
# Initialize Quantum Autoencoder (compress 4 dimensions to 2)
qae = QuantumAutoencoder(n_input_qubits=4, n_latent_qubits=2, n_layers=2)

print("Quantum Autoencoder Configuration:")
print(f"Input qubits: {qae.n_input}")
print(f"Latent qubits (compressed): {qae.n_latent}")
print(f"Trash qubits: {qae.n_trash}")
print(f"Number of trainable parameters: {len(qae.params)}")
print(f"Compression ratio: {qae.n_latent / qae.n_input:.1%}")

# Note: Full training would require quantum circuit execution
# This demonstrates the initialization and structure
print("\n✓ Quantum Autoencoder initialized successfully")

## 5. Quantum Support Vector Classifier (QSVC)

**Description**: A quantum kernel-based support vector classifier that maps data into a high-dimensional Hilbert space where complex classes become linearly separable. Particularly effective for non-linear classification problems.

**Use Case**: Classify iris flowers into their species using quantum kernel advantages.

In [ ]:
# Prepare binary classification data (Setosa vs others)
X_clf = iris_df.iloc[:100, :-1].values
y_clf = (iris_df.iloc[:100, -1] == 'Setosa').astype(int).values

# Normalize features to [0, 1]
X_clf = (X_clf - X_clf.min(axis=0)) / (X_clf.max(axis=0) - X_clf.min(axis=0))

# Split data
split_idx = 80
X_train_clf = X_clf[:split_idx]
y_train_clf = y_clf[:split_idx]
X_test_clf = X_clf[split_idx:]
y_test_clf = y_clf[split_idx:]

# Initialize and train Quantum SVC
qsvc = QuantumSVC(n_qubits=4, C=1.0)
qsvc.fit(X_train_clf, y_train_clf)

# Make predictions
predictions_qsvc = qsvc.predict(X_test_clf)

print("Quantum SVC Classification Results:")
print(f"Training samples: {len(X_train_clf)}")
print(f"Test samples: {len(X_test_clf)}")
print(f"Accuracy: {(predictions_qsvc == y_test_clf).mean():.2%}")
print(f"First 10 predictions: {predictions_qsvc[:10]}")

## 6. Quantum K-Means Clustering

**Description**: K-Means clustering that uses a quantum kernel to estimate distances in Hilbert space. Points are clustered based on their quantum similarity rather than Euclidean distance.

**Use Case**: Discover natural clusters in the iris dataset based on quantum similarity.

In [ ]:
# Prepare clustering data
X_cluster = iris_df.iloc[:50, :-1]  # Use first 50 samples
X_cluster = (X_cluster - X_cluster.min()) / (X_cluster.max() - X_cluster.min())

# Initialize and fit Quantum K-Means
qkm = QuantumKMeans(n_clusters=3, n_qubits=4, max_iter=5)
qkm.fit(X_cluster)

# Get cluster labels
cluster_labels = qkm.labels_

print("Quantum K-Means Clustering Results:")
print(f"Number of clusters: {qkm.n_clusters}")
print(f"Cluster distribution:")
for i in range(qkm.n_clusters):
    print(f"  Cluster {i}: {(cluster_labels == i).sum()} samples")
print(f"\nCentroids shape: {qkm.centroids_.shape}")

## 7. Quantum Kernel Regressor

**Description**: Performs regression (predicting continuous values) using a quantum kernel. Maps data implicitly to high-dimensional Hilbert space to capture non-linear patterns.

**Use Case**: Predict sepal length from other iris measurements using quantum kernel regression.

In [ ]:
# Prepare regression data (predict sepal length from other features)
X_reg = iris_df.iloc[:100, 1:].values  # sepal.width, petal.length, petal.width
y_reg = iris_df.iloc[:100, 0].values   # sepal.length

# Normalize features
X_reg = (X_reg - X_reg.min(axis=0)) / (X_reg.max(axis=0) - X_reg.min(axis=0))
y_reg = (y_reg - y_reg.min()) / (y_reg.max() - y_reg.min())

# Split data
split_idx = 80
X_train_reg = X_reg[:split_idx]
y_train_reg = y_reg[:split_idx]
X_test_reg = X_reg[split_idx:]
y_test_reg = y_reg[split_idx:]

# Initialize and train Quantum Kernel Regressor
qkr = QuantumKernelRegressor(n_qubits=3, alpha=0.1)
qkr.fit(X_train_reg, y_train_reg)

# Make predictions
predictions_qkr = qkr.predict(X_test_reg)

print("Quantum Kernel Regressor Results:")
print(f"Training samples: {len(X_train_reg)}")
print(f"Test samples: {len(X_test_reg)}")
print(f"Mean Squared Error: {((predictions_qkr - y_test_reg)**2).mean():.4f}")
print(f"First 5 predictions: {predictions_qkr[:5]}")
print(f"First 5 actual values: {y_test_reg[:5]}")

## 8. Quantum Feature Selector

**Description**: Selects an optimal subset of features using a quantum-inspired combinatorial optimization approach. Maximizes relevance to the target while minimizing redundancy between features.

**Use Case**: Select the most important iris measurements for species classification.

In [ ]:
# Prepare feature selection data
X_feature = iris_df.iloc[:, :-1].copy()
y_feature = (iris_df.iloc[:, -1] == 'Setosa').astype(int)

# Initialize and fit Feature Selector (select 2 most important features)
qfs = QuantumFeatureSelector(n_features_to_select=2)
qfs.fit(X_feature, y_feature)

# Get selected features
selected_features = qfs.selected_features_

print("Quantum Feature Selector Results:")
print(f"Original features: {list(X_feature.columns)}")
print(f"Selected features: {selected_features}")
print(f"Reduction: {len(selected_features)}/{len(X_feature.columns)} features")

# Transform data using selected features
X_transformed = qfs.transform(X_feature)
print(f"\nTransformed data shape: {X_transformed.shape}")

## 9. Variational Quantum Eigensolver (VQE)

**Description**: VQE finds the ground state energy of a Hamiltonian by optimizing variational circuit parameters. Essential for quantum chemistry simulations and optimization problems.

**Use Case**: Find the minimum energy eigenvalue of a sample Hamiltonian matrix.

In [ ]:
# Create a simple Hamiltonian matrix from iris data
# Extract first 4x4 covariance matrix
X_vqe = iris_df.iloc[:100, :-1].values
H_matrix = np.cov(X_vqe[:, :4].T)

print("Hamiltonian Matrix (from iris data covariance):")
print(H_matrix)

# Initialize and train VQE
vqe = VariationalQuantumEigensolver(
    backend_name="aer_simulator",
    shots=1024,
    ansatz_depth=3,
    learning_rate=0.01,
    max_iterations=50
)
vqe.fit(H_matrix)

# Get the minimum energy found
min_energy = vqe.predict()

print(f"\nVQE Results:")
print(f"Minimum energy found: {min_energy[0]:.6f}")
print(f"Ground state energy (true): {np.min(np.linalg.eigvalsh(H_matrix)):.6f}")
print(f"Optimization history (last 5 iterations):")
print(vqe.history[-5:])

## 10. Quantum Neural Network

**Description**: A hybrid quantum-classical neural network that combines quantum circuits with classical neural network training. Learns non-linear transformations through parameter optimization.

**Use Case**: Train a quantum neural network for iris flower classification.

In [ ]:
# Prepare neural network data
X_nn = iris_df.iloc[:100, :-1].values
y_nn = (iris_df.iloc[:100, -1] == 'Setosa').astype(float).values

# Normalize features
X_nn = (X_nn - X_nn.min(axis=0)) / (X_nn.max(axis=0) - X_nn.min(axis=0))

# Split data
split_idx = 80
X_train_nn = X_nn[:split_idx]
y_train_nn = y_nn[:split_idx]
X_test_nn = X_nn[split_idx:]
y_test_nn = y_nn[split_idx:]

# Initialize and train Quantum Neural Network
qnn = QuantumNeuralNetwork(
    backend_name="aer_simulator",
    shots=1024,
    n_qubits=4,
    layers=2,
    learning_rate=0.01
)
qnn.fit(X_train_nn, y_train_nn, epochs=10)

# Make predictions
predictions_qnn = qnn.predict(X_test_nn)

print("Quantum Neural Network Results:")
print(f"Training epochs: 10")
print(f"Final training loss: {qnn.training_history[-1]:.4f}")
print(f"First 10 predictions: {predictions_qnn[:10]}")
print(f"Predictions in range [0, 1]: {np.all((predictions_qnn >= -1) & (predictions_qnn <= 1))}")

## 11. Quantum Ensemble

**Description**: Combines multiple quantum models with a voting mechanism. Improves prediction accuracy and robustness by aggregating predictions from diverse models.

**Use Case**: Build an ensemble of quantum models for more reliable iris classification.

In [ ]:
# Prepare ensemble data
X_ensemble = iris_df.iloc[:100, :-1].values
y_ensemble = (iris_df.iloc[:100, -1] == 'Setosa').astype(int).values

# Normalize features
X_ensemble = (X_ensemble - X_ensemble.min(axis=0)) / (X_ensemble.max(axis=0) - X_ensemble.min(axis=0))

# Split data
split_idx = 80
X_train_ensemble = X_ensemble[:split_idx]
y_train_ensemble = y_ensemble[:split_idx]
X_test_ensemble = X_ensemble[split_idx:]
y_test_ensemble = y_ensemble[split_idx:]

# Initialize and train Quantum Ensemble
qe = QuantumEnsemble(
    backend_name="aer_simulator",
    shots=1024,
    n_models=3,
    voting="majority"
)
qe.fit(X_train_ensemble, y_train_ensemble)

# Make predictions
predictions_ensemble = qe.predict(X_test_ensemble)

print("Quantum Ensemble Results:")
print(f"Number of models in ensemble: {qe.n_models}")
print(f"Voting strategy: {qe.voting}")
print(f"Model weights: {qe.weights}")
print(f"First 10 ensemble predictions: {predictions_ensemble[:10]}")

## 12. Quantum Time Series Predictor

**Description**: A quantum model designed for time series prediction. Uses quantum circuits to learn temporal patterns and predict future values based on historical data.

**Use Case**: Predict future values in a time series derived from iris measurements.

In [ ]:
# Create a time series from iris measurements
# Use sepal length measurements as a time series
time_series_data = iris_df['sepal.length'].values[:50]
time_series_data = (time_series_data - time_series_data.mean()) / time_series_data.std()

print("Time Series Data (first 10 values):")
print(time_series_data[:10])

# Initialize and train Quantum Time Series Predictor
qtsp = QuantumTimeSeriesPredictor(
    backend_name="aer_simulator",
    shots=1024,
    lookback=5,
    horizon=1
)
qtsp.fit(time_series_data, epochs=10)

# Predict next values
predictions_ts = qtsp.predict(time_series_data[:20], n_steps=5)

print(f"\nQuantum Time Series Predictor Results:")
print(f"Lookback window: {qtsp.lookback}")
print(f"Prediction horizon: {qtsp.horizon}")
print(f"Predicted next 5 values: {predictions_ts}")
print(f"\nLast 10 actual values:")
print(time_series_data[:-10:-1][::-1])

## Summary

This tutorial demonstrated all major tools in the QDET analytics module:

1. **QuantumKernelAnomalyDetector** - Detects anomalies using quantum kernels and One-Class SVM
2. **QuantumAutoencoder** - Compresses high-dimensional data into lower-dimensional quantum representations
3. **QuantumSVC** - Quantum Support Vector Classifier for binary classification using quantum kernels
4. **QuantumKMeans** - Clustering using quantum distance metrics in Hilbert space
5. **QuantumKernelRegressor** - Non-linear regression with quantum kernel methods
6. **QuantumFeatureSelector** - Selects optimal feature subsets using quantum-inspired optimization
7. **VariationalQuantumEigensolver (VQE)** - Finds ground state energies of Hamiltonians
8. **QuantumNeuralNetwork** - Hybrid quantum-classical neural networks for complex learning tasks
9. **QuantumEnsemble** - Combines multiple quantum models for improved robustness
10. **QuantumTimeSeriesPredictor** - Predicts future values in time series data

Each tool leverages quantum computing principles to handle high-dimensional data and non-linear patterns more effectively than classical methods.